In [1]:
import os
import warnings
import requests
import statistics
import json
from tqdm import tqdm
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

tqdm.pandas()
warnings.filterwarnings("ignore")

In [2]:
output_path = "./student_grades.json"

force_request = 0
session_id = "fmrb5gkzz2q55ilno0nejr4b"

In [3]:
def grab_sysgen(session_id, ashoka_email):
    url = "https://ams.ashoka.edu.in/Contents/Masters/ViewstudentDirectory.aspx/GetStudentList"
    headers = {
        "Host": "ams.ashoka.edu.in",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "en-GB,en-US;q=0.7,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "Content-Type": "application/json; charset=utf-8",
        "X-Requested-With": "XMLHttpRequest",
        "Content-Length": "260",
        "Origin": "https://ams.ashoka.edu.in",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": "https://ams.ashoka.edu.in/Contents/Masters/ViewstudentDirectory.aspx",
        "Cookie": "ASP.NET_SessionId=" + session_id + "; __AJAXAntiXsrfToken=pqqFQQCNulHpSROVS20HjZGCxmhQksGUTzLu756vcdAjU5gAra9hA_6GHr34sMZPXJwwlNjwhgvJxZ7-xCGziZ8QAVPjiljAXeo1ALMW4T01",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
    }
    post_data = json.dumps(
        {
            "XMLData": "<tbl><tr><ProgrammeSysGenId></ProgrammeSysGenId><SessionSysGenId"
                       "></SessionSysGenId "
                       "><ScheduleSysGenId></ScheduleSysGenId><CourseSysGenId></CourseSysGenId><LSNo></LSNo><DSNo"
                       "></DSNo><SearchByText>" + ashoka_email + "</SearchByText></tr></tbl>",
            "ListType": "All",
        }
    )

    try:
        response = requests.post(url, data=post_data, headers=headers)
        return str(response.content).split('\\\\"UserSysGenId\\\\": \\\\"')[1][0:11]
    except:
        return "0000000000"

In [4]:
def grab_grade(session_id, ashoka_email, sysgen_id, force_request, file_name):
    if not force_request:
        with open(file_name, 'r') as file:
            file_objects = json.load(file)
            for student_index, student_object in enumerate(file_objects["data"]):
                if ashoka_email in student_object["AshokaEmailID"] or ashoka_email in student_object["Name"]:
                    print("Found in file!")
                    return student_object

    print(f"Sending request for ID {sysgen_id}, email {ashoka_email}...")
    url = "https://ams.ashoka.edu.in/Services/MasterServices.asmx/GetCourseReport"

    headers = {
        "Host": "ams.ashoka.edu.in",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "en-GB,en-US;q=0.7,en;q=0.3",
        "Accept-Encoding": "gzip, deflate, br",
        "Content-Type": "application/json; charset=utf-8",
        "X-Requested-With": "XMLHttpRequest",
        "Content-Length": "260",
        "Origin": "https://ams.ashoka.edu.in",
        "DNT": "1",
        "Connection": "keep-alive",
        "Referer": "https://ams.ashoka.edu.in/Contents/CourseManagement/StudentCourseReport_Student.aspx",
        "Cookie": "ASP.NET_SessionId=" + session_id + "; __AJAXAntiXsrfToken=_5B4rluYFBXxsvnk7hMJt1rk8slaVMdzjvFRVxU91q6nO2Bx7tKqUKHVxYUHtyJlDDOh2oktIyVR_xwJBM2H6E8xtWvhNOFoqrn-_1pcVOg",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "Sec-GPC": "1",
    }
    post_data = json.dumps(
        {"xml": "<tbl><tr><UserSysGenId>" + sysgen_id + "</UserSysGenId></tr></tbl>"}
    )

    response = requests.post(url, data=post_data, headers=headers)

    return (
        parse_grade(response.content.replace(b"\\r", b"").replace(b"\\n", b"").replace(b'\\"', b'"'))
    )

In [5]:
def parse_grade(json_string):
    json_data = {}
    json_string = str(json_string)[2:]
    if "ProgrammeSysGenId" not in json_string:
        return None

    json_data["ProgrammeSysGenId"] = json_string.split('ProgrammeSysGenId": "')[1].split('",      "Name"')[0]
    json_data["Name"] = json_string.split('Name": "')[1].split('",      "CurrentCGPA"')[0]
    json_data["CurrentCGPA"] = json_string.split('CurrentCGPA": "')[1].split('",      "MajorCGPA"')[0]
    json_data["MajorCGPA"] = json_string.split('MajorCGPA": "')[1].split('",      "OVERALL_CREDITS"')[0]
    json_data["AshokaID"] = json_string.split('AshokaId": "')[1].split('",      "AshokaEmailId"')[0]
    json_data["AshokaEmailID"] = json_string.split('AshokaEmailId": "')[1].split('",      "ProgrammeSysGenId"')[0]

    semesters = json_string.split('"CourseCode"')[0].split('"Semester":')[1:-1]
    json_data["semesters"] = []
    for semester in semesters:
        semester_json = {}
        semester_json["semester"] = (
            semester.split(' "')[1].split('",      "Status')[0].strip('",     ')
        )
        semester_json["GPA"] = float(
            semester.split('"GPA": "')[1].split('",      "SemesterCGPA')[0]
        )
        semester_json["CGPA"] = float(
            semester.split('"SemesterCGPA": "')[1].split('",      "Deans_List"')[0]
        )
        semester_json["Dean's List"] = (
            True
            if "Dean"
               in semester.split('"Deans_List": "')[1].split(
                '"    },    {      "ScheduleSysGenId'
            )[0]
            else False
        )
        json_data["semesters"].append(semester_json)

    courses = json_string.split('"Table3": [')[1].split("ScheduleSysGenId")[1:]
    json_data["courses"] = []
    for course in courses:
        course_json = {}
        course_json["CourseSysGenId"] = course.split('"CourseSysGenId": "')[1].split(
            '",      "Semester"'
        )[0]
        course_json["semester"] = course.split('"Semester": "')[1].split(
            '",      "CourseCode"'
        )[0]
        course_json["code"] = course.split('"CourseCode": "')[1].split(
            '",      "EquivalentCourseCode"'
        )[0]
        course_json["credits"] = course.split('"Credits": ')[1].split(
            ',      "GradePoint"'
        )[0]
        course_json["name"] = course.split('"CourseName": "')[1].split(
            '",      "Grade'
        )[0]
        course_json["grade"] = course.split('"Grade": "')[1].split(
            '",      "CountinCredits"'
        )[0]
        json_data["courses"].append(course_json)

    return json_data

In [6]:
def update_file(file_name, new_object):
    with open(file_name, 'r') as file:
        file_objects = json.load(file)

    for student_index, student_object in enumerate(file_objects["data"]):

        if new_object["AshokaEmailID"] == student_object["AshokaEmailID"]:
            if student_object == new_object:
                return "Exists in file!"

            else:
                file_objects["data"][student_index] = new_object
                with open(file_name, 'w') as file:
                    json.dump(file_objects, file, indent=4)
                return "Updated in file!"

    else:
        file_objects["data"].append(new_object)
        with open(file_name, 'w') as file:
            json.dump(file_objects, file, indent=4)
        return "Appended to file!"

In [7]:
def update_all(session_id, force_request, file_name):
    with open(file_name, 'r') as file:
        file_objects = json.load(file)

    for student_index, student_object in enumerate(file_objects["data"]):
        student_sysgen = grab_sysgen(session_id, student_object["AshokaEmailID"])
        student_grades = grab_grade(session_id, student_object["AshokaEmailID"], student_sysgen, force_request, file_name)
        print(update_file(file_name, student_grades))

In [8]:
email = "aaditya.shetty@ashoka.edu.in"

student_sysgen = grab_sysgen(session_id, email)
student_grades = grab_grade(session_id, email, student_sysgen, force_request, output_path)
pprint(student_grades)
print(update_file(output_path, student_grades))
# update_all(session_id, force_request, file_name)

Found in file!
{'AshokaEmailID': 'aaditya.shetty@ashoka.edu.in',
 'AshokaID': 'UG-14-0937',
 'CurrentCGPA': '3.36',
 'MajorCGPA': '3.68',
 'Name': 'Aaditya Shetty',
 'ProgrammeSysGenId': 'PRG00000001',
 'courses': [{'CourseSysGenId': 'CRS00000002',
              'code': 'CT-001',
              'credits': '4',
              'grade': 'B-',
              'name': 'Introduction to Critical Thinking',
              'semester': 'Monsoon 2014'},
             {'CourseSysGenId': 'CRS00000017',
              'code': 'FC-0502',
              'credits': '4',
              'grade': 'A-',
              'name': 'Literature and the World',
              'semester': 'Monsoon 2014'},
             {'CourseSysGenId': 'CRS00000018',
              'code': 'FC-0503',
              'credits': '4',
              'grade': 'A-',
              'name': 'Mind and Behaviour',
              'semester': 'Monsoon 2014'},
             {'CourseSysGenId': 'CRS00000021',
              'code': 'FC-0504',
              'credi

In [9]:
def generate_ranking(file_name):
    with open(file_name, 'r') as file:
        students = json.load(file)["data"]

    pairs = []
    for student in students:
        if "_ug" in student["AshokaEmailID"] or "" in [x["name"] for x in student["courses"]]:
            pairs.append((student["CurrentCGPA"], student["Name"]))

    data = [float(x[0]) for x in pairs]
    print(f"Mean: {statistics.mean(data):.2f}")
    print(f"Median: {statistics.median(data):.2f}")
    print(f"Mode: {statistics.mode(data):.2f}")
    print(f"Std Dev: {statistics.stdev(data):.2f}")
    print(f"Variance: {statistics.variance(data):.2f}")

    return sorted(pairs, reverse=True, key=lambda x: x[0])

In [10]:
ranking = generate_ranking(output_path)
pprint([(ranking.index(x)+1, x) for x in ranking])

Mean: 3.22
Median: 3.43
Mode: 0.00
Std Dev: 0.74
Variance: 0.55
[(1, ('4.00', 'Debanshi Chatterjee')),
 (2, ('4.00', 'Meesha Chotai')),
 (3, ('4.00', 'Riya Jain')),
 (4, ('3.99', 'Aishwarya Raviganesh')),
 (5, ('3.99', 'Pankhi Mehta')),
 (6, ('3.98', 'Akila Ranganathan')),
 (7, ('3.98', 'Damini Jain')),
 (8, ('3.98', 'Sargam Mitra')),
 (9, ('3.98', 'Sudarshan Ramanujam Sriram Aittreya')),
 (10, ('3.98', 'Sidharth Sameer Wagle')),
 (11, ('3.98', 'Tejas Nageshwaran')),
 (12, ('3.97', 'Saloni Mehta')),
 (13, ('3.97', 'Vighnesh H J')),
 (14, ('3.97', 'Daksh Walia')),
 (15, ('3.97', 'Paarthvi Raj Singh')),
 (16, ('3.97', 'Rohan Chopra')),
 (17, ('3.97', 'Hiba Hasan')),
 (18, ('3.97', 'Mohan Rajagopal')),
 (19, ('3.97', 'Namya Mitra')),
 (20, ('3.97', 'Vineet Karlapalem')),
 (21, ('3.97', 'Aarushi Verma')),
 (22, ('3.97', 'Aditi Ponnammal')),
 (23, ('3.97', 'Isha Lokre')),
 (24, ('3.97', 'Teesha Rishi Aurora')),
 (25, ('3.96', 'Kabir Singh Bakshi')),
 (26, ('3.96', 'Niyati Sanjay Bafna')),
 

In [11]:
def generate_course_grades(file_name, course_name, semester):
    with open(file_name, 'r') as file:
        file_objects = json.load(file)

    grades = []
    for student in file_objects["data"]:
        for course in student["courses"]:

            if course_name in course["name"] and semester in course["semester"]:
                grades.append((course["grade"], student["Name"], course["semester"]))
                break

    map = {'A': 4, 'A-': 3.7, 'B+': 3.3, 'B': 3, 'B-': 2.7, 'C+': 2.3, 'C': 2, 'C-': 1.7, 'D+': 1.3, 'D': 1, 'D-': 0.7, 'F': 0, 'WF': 0}
    data = [map[x[0]] for x in grades if x[0] not in ['', 'P', 'TP', 'TC', 'W', 'I', 'AU', 'WAU', 'WX']]
    print(f"Mean: {statistics.mean(data):.2f}")
    print(f"Median: {statistics.median(data):.2f}")
    print(f"Mode: {statistics.mode(data):.2f}")
    print(f"Std Dev: {statistics.stdev(data):.2f}")
    print(f"Variance: {statistics.variance(data):.2f}")

    return sorted(grades, reverse=False, key=lambda x: x[0])

In [15]:
generate_course_grades(output_path, "Natural Language Processing", "")

Mean: 2.60
Median: 2.70
Mode: 3.30
Std Dev: 0.75
Variance: 0.56


[('A', 'Kahaan Harsh Shah', 'Spring 2024'),
 ('A', 'Rithik Kumar S', 'Spring 2024'),
 ('A-', 'Gautam Ahuja', 'Spring 2024'),
 ('B', 'Aryan Yadav', 'Spring 2024'),
 ('B', 'Medini Chopra', 'Spring 2024'),
 ('B', 'Nishtha Bimal Das', 'Spring 2024'),
 ('B', 'Samvit Jatia', 'Spring 2024'),
 ('B', 'Nistha Singh', 'Spring 2024'),
 ('B+', 'Aaryan Shah', 'Spring 2024'),
 ('B+', 'Karan Kumar', 'Spring 2024'),
 ('B+', 'Maanas Kejriwal', 'Spring 2024'),
 ('B+', 'Soumyajit Basu', 'Spring 2024'),
 ('B+', 'Suyog Yogesh Joshi', 'Spring 2024'),
 ('B+', 'Vidur Kaushik', 'Spring 2024'),
 ('B-', 'Abhimanyu Sharma', 'Spring 2024'),
 ('B-', 'Apoorva Arora', 'Spring 2024'),
 ('B-', 'Aaryan Nagpal', 'Spring 2024'),
 ('B-', 'Devashish Vij', 'Spring 2024'),
 ('B-', 'Karthik Sridhar', 'Spring 2024'),
 ('C', 'Elijah Pendse Samuel', 'Spring 2024'),
 ('C', 'Muskaan Chugh', 'Spring 2024'),
 ('C', 'Jigyansu Rout', 'Spring 2024'),
 ('C', 'Rahul Navnath Gajare', 'Spring 2024'),
 ('C', 'Santosh Adhikari', 'Spring 2024')